In [1]:
from ns3gym import ns3env
import subprocess
from collections import deque
import numpy as np

from agents.teacher import Teacher

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
simTime = 10 # seconds
stepTime = 0.1  # seconds

EPISODE_COUNT = 30
steps_per_ep = int(simTime/stepTime)

SCRIPT_RUNNING = False

script_exec_command = f'../../waf --run "linear-mesh --simTime={simTime} --envStepTime={stepTime}"'
print("Steps per episode:", steps_per_ep)

Steps per episode: 100


In [3]:
port = 5555
seed = 0

simArgs = {"--simTime": simTime,
           "--testArg": 123,
           "--nodeNum": 5,
           "--distance": 500}
debug = False

subprocess.Popen(['bash', '-c', script_exec_command])
SCRIPT_RUNNING = True

env = ns3env.Ns3Env(port=port, stepTime=stepTime, startSim=0, simSeed=seed, simArgs=simArgs, debug=debug)

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


In [4]:
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

Observation space shape: Box(10,)
Action space shape: Box(1,)


In [5]:
class Agent:
    def __init__(self, action_space):
        self.action_space = action_space
        self.actor_loss = 0
        self.critic_loss = 0
        
    def act(self, *args):
        return [self.action_space.sample()]
    
    def step(self, *args):
        pass
    
    def reset(self):
        pass

In [6]:
agent = Agent(env.action_space)
teacher = Teacher(agent, env, 1)
teacher.train(EPISODE_COUNT, simTime, stepTime, "BEB")

COMET INFO: old comet version (1.0.45) detected. current: 1.0.46 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/rl-in-wifi/afc3d6ae216f44f7872e84f7d8ddf53f

100%|█████████████████████████████████████████████████████████████| 100/100 [01:09<00:00,  1.44it/s, mb_sent=383.92 Mb]


Sent 383.92 Mb/s.	Mean speed: 38.39 Mb/s	Episode 1/30 finished

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


100%|█████████████████████████████████████████████████████████████| 100/100 [01:05<00:00,  1.52it/s, mb_sent=383.92 Mb]


Sent 383.92 Mb/s.	Mean speed: 38.39 Mb/s	Episode 2/30 finished

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


100%|█████████████████████████████████████████████████████████████| 100/100 [01:05<00:00,  1.52it/s, mb_sent=383.92 Mb]


...

Sent 383.92 Mb/s.	Mean speed: 38.39 Mb/s	Episode 29/30 finished

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


100%|█████████████████████████████████████████████████████████████| 100/100 [01:05<00:00,  1.54it/s, mb_sent=383.92 Mb]


Sent 383.92 Mb/s.	Mean speed: 38.39 Mb/s	Episode 30/30 finished



COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Metrics:
COMET INFO:             Cumulative reward: 1.1887695327459369
COMET INFO:                Megabytes sent: 383.915764
COMET INFO:                 Per-ep reward: 1.1887695327459369
COMET INFO:          Round megabytes sent: 3.842285
COMET INFO:                  Round reward: 0.01230468787252903
COMET INFO:                         Speed: 38.391576400000005
COMET INFO:                    actor_loss: 0
COMET INFO:                   critic_loss: 0
COMET INFO:         sys.gpu.0.free_memory: 9546915840
COMET INFO:     sys.gpu.0.gpu_utilization: 1
COMET INFO:        sys.gpu.0.total_memory: 11811160064
COMET INFO:         sys.gpu.0.used_memory: 2264244224
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


Training finished.
